| Model                  | Full Precision | Full Recall | Full F1 | Full Accuracy | Test Precision | Test Recall | Test F1 | Test Accuracy |
| ---------------------- | -------------- | ----------- | ------- | ------------- | -------------- | ----------- | ------- | ------------- |
| predict_simple         | 0.59           | 0.97        | 0.73    | 0.64          | 0.61           | 0.95        | 0.74    | 0.65          |
| predict_cot            | 0.75           | 0.59        | 0.66    | 0.68          | 0.85           | 0.56        | 0.68    | 0.71          |
| predict_subcategory_v1 | 0.75           | 0.67        | 0.71    | 0.72          | 0.90           | 0.63        | 0.74    | 0.77          |
| predict_subcategory_v2 | 0.78           | 0.65        | 0.71    | 0.73          | 0.82           | 0.78        | 0.80    | 0.79          |
| predict_subcategory_v3 | 0.74           | 0.73        | 0.73    | 0.73          | 0.76           | 0.68        | 0.72    | 0.71          |
| predict_deontology     | 0.76           | 0.71        | 0.73    | 0.74          | 0.68           | 0.68        | 0.68    | 0.66          |
| predict_cards_v1       | 0.72           | 0.75        | 0.73    | 0.73          | 0.61           | 0.93        | 0.74    | 0.65          |
| predict_cards_v2       | 0.57           | 0.91        | 0.70    | 0.62          | 0.65           | 0.95        | 0.77    | 0.70          |


In [1]:
import asyncio
import json
import sys
from typing import Callable, Literal

import pandas as pd
from litellm import acompletion
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm

sys.path.append("../..")

from climateguard.evaluation import (
    plot_binary_confusion_matrix,
    show_classification_report,
)

# Limit concurrency
semaphore = asyncio.Semaphore(150)

In [2]:
def load_dataset(filepath: str) -> pd.DataFrame:
    df = pd.read_csv(filepath).set_index("id")
    df = df[["text", "Misinfo"]].rename(columns={"Misinfo": "misinformation"})
    df["misinformation"] = df["misinformation"].astype(bool)
    return df


train = load_dataset("../../data/charlotte/train_227.csv")
test = load_dataset("../../data/charlotte/test_77.csv")

In [3]:
async def evaluate_prompt_on_one_split(
    predict: Callable[[str], bool], df: pd.DataFrame
) -> None:
    # Avoid in-place modifications
    df = df.copy()
    # Run predictions
    df["misinformation_pred"] = await tqdm.gather(
        *[predict(text) for text in df["text"]]
    )
    # Show metrics
    print()
    show_classification_report(df["misinformation"], df["misinformation_pred"])
    plot_binary_confusion_matrix(df["misinformation"], df["misinformation_pred"])


async def evaluate(
    predict: Callable[[str], bool], train: pd.DataFrame, test: pd.DataFrame
) -> None:
    print("TEST\n====\n")
    await evaluate_prompt_on_one_split(predict, test)
    print("FULL\n====\n")
    await evaluate_prompt_on_one_split(predict, pd.concat([train, test]))

In [4]:
class ClassificationSimple(BaseModel):
    answer: Literal["oui", "non"] = Field(
        alias="réponse",
        description='"oui" si l\'extrait contient de la désinformation sur le climat, sinon "non"',
    )


async def predict_simple(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""D'après toi, cet extrait radio contient-il de la désinformation sur le climat ? (oui/non)

{text}""",
            }
        ],
        response_format=ClassificationSimple,
        timeout=60,
        max_tokens=1000,
        temperature=0,
    )
    classification = ClassificationSimple(
        **json.loads(response.choices[0].message.content)
    )
    return classification.answer == "oui"


await evaluate(predict_simple, train, test)

TEST
====



100%|██████████| 77/77 [00:03<00:00, 20.91it/s]


              precision    recall  f1-score   support

       False       0.85      0.31      0.45        36
        True       0.61      0.95      0.74        41

    accuracy                           0.65        77
   macro avg       0.73      0.63      0.60        77
weighted avg       0.72      0.65      0.61        77



FULL
====



100%|██████████| 304/304 [00:09<00:00, 32.57it/s] 


              precision    recall  f1-score   support

       False       0.90      0.31      0.46       151
        True       0.59      0.97      0.73       153

    accuracy                           0.64       304
   macro avg       0.75      0.64      0.60       304
weighted avg       0.74      0.64      0.60       304



In [5]:
class ClassificationCoT(BaseModel):
    reasoning: str = Field(
        alias="raisonnement", description="Ton raisonnement pour classifier l'extrait"
    )
    answer: Literal["oui", "non"] = Field(
        alias="réponse",
        description='"oui" si l\'extrait contient de la désinformation sur le climat, sinon "non"',
    )


async def predict_cot(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""D'après toi, cet extrait radio contient-il de la désinformation sur le climat ? (oui/non)
Éxplique ton raisonnement étape par étape.

{text}""",
            }
        ],
        response_format=ClassificationCoT,
        timeout=60,
        max_tokens=1000,
        temperature=0,
    )
    classification = ClassificationCoT(
        **json.loads(response.choices[0].message.content)
    )
    return classification.answer == "oui"


await evaluate(predict_cot, train, test)

TEST
====



  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:10<00:00,  7.40it/s]


              precision    recall  f1-score   support

       False       0.62      0.78      0.69        36
        True       0.75      0.59      0.66        41

    accuracy                           0.68        77
   macro avg       0.69      0.68      0.67        77
weighted avg       0.69      0.68      0.67        77



FULL
====



100%|██████████| 304/304 [00:15<00:00, 20.15it/s] 


              precision    recall  f1-score   support

       False       0.70      0.77      0.73       151
        True       0.75      0.67      0.71       153

    accuracy                           0.72       304
   macro avg       0.72      0.72      0.72       304
weighted avg       0.72      0.72      0.72       304



In [6]:
class ClassificationSubcategoryV1(BaseModel):
    reasoning: str = Field(
        alias="raisonnement",
        description="Raisonnement étape par étape justifiant la classification de l'extrait",
    )
    subcategory: Literal[
        "Désinformation",
        "Paroles rapportées",
        "Oeuvre de fiction",
        "Rien à signaler",
    ] = Field(alias="sous-catégorie")
    category: Literal["Désinformation", "Pas de désinformation"] = Field(
        alias="catégorie"
    )


async def predict_subcategory_v1(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Vous êtes un assistant IA spécialisé dans l'analyse de contenu médiatique pour détecter la désinformation sur le climat. Votre tâche est d'examiner un extrait textuel de 2 minutes provenant d'une émission télévisée ou radiophonique et de le classer selon la présence ou l'absence de désinformation.

## Extrait à analyser

{text.strip()}

## Classez l'extrait dans l'une des catégories suivantes

Désinformation : L'affirmation est factuellement inexacte, déformée ou exagérée.

Pas de Désinformation :
- Rien à signaler : Aucune désinformation sur le climat n'est présente.
- Paroles rapportées : L'extrait rapporte fidèlement les propos de quelqu'un d'autre.
- Oeuvre de fiction : L'extrait provient d'une oeuvre de fiction.""",
            }
        ],
        response_format=ClassificationSubcategoryV1,
        timeout=60,
        max_tokens=1000,
        temperature=0,
    )

    classification = ClassificationSubcategoryV1(
        **json.loads(response.choices[0].message.content)
    )
    return classification.category == "Désinformation"


await evaluate(predict_subcategory_v1, train, test)

TEST
====



  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:07<00:00, 10.00it/s]


              precision    recall  f1-score   support

       False       0.70      0.92      0.80        36
        True       0.90      0.66      0.76        41

    accuracy                           0.78        77
   macro avg       0.80      0.79      0.78        77
weighted avg       0.81      0.78      0.78        77



FULL
====



100%|██████████| 304/304 [00:15<00:00, 19.96it/s] 


              precision    recall  f1-score   support

       False       0.70      0.81      0.75       151
        True       0.78      0.65      0.71       153

    accuracy                           0.73       304
   macro avg       0.74      0.73      0.73       304
weighted avg       0.74      0.73      0.73       304



In [7]:
class ClassificationSubcategoryV2(BaseModel):
    reasoning: str = Field(
        alias="raisonnement",
        description="Raisonnement étape par étape justifiant la classification de l'extrait",
    )
    subcategory: Literal[
        "Faux",
        "Trompeur",
        "Paroles rapportées",
        "Oeuvre de fiction",
        "Rien à signaler",
    ] = Field(alias="sous-catégorie")
    category: Literal["Désinformation", "Pas de désinformation"] = Field(
        alias="catégorie"
    )


async def predict_subcategory_v2(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Vous êtes un assistant IA spécialisé dans l'analyse de contenu médiatique pour détecter la désinformation sur le climat. Votre tâche est d'examiner un extrait textuel de 2 minutes provenant d'une émission télévisée ou radiophonique et de le classer selon la présence ou l'absence de désinformation.

## Extrait à analyser

{text.strip()}

## Classez l'extrait dans l'une des catégories suivantes

Désinformation :
- Faux : L'affirmation est factuellement inexacte.
- Trompeur : L'affirmation repose sur un élément factuel mais est déformée ou exagérée.

Pas de Désinformation :
- Rien à signaler : Aucune désinformation sur le climat n'est présente.
- Paroles rapportées : L'extrait rapporte fidèlement les propos de quelqu'un d'autre.
- Oeuvre de fiction : L'extrait provient d'une oeuvre de fiction.""",
            }
        ],
        response_format=ClassificationSubcategoryV2,
        timeout=60,
        max_tokens=1000,
        temperature=0,
    )

    classification = ClassificationSubcategoryV2(
        **json.loads(response.choices[0].message.content)
    )
    return classification.category == "Désinformation"


await evaluate(predict_subcategory_v2, train, test)

TEST
====



100%|██████████| 77/77 [00:18<00:00,  4.08it/s]


              precision    recall  f1-score   support

       False       0.74      0.81      0.77        36
        True       0.82      0.76      0.78        41

    accuracy                           0.78        77
   macro avg       0.78      0.78      0.78        77
weighted avg       0.78      0.78      0.78        77



FULL
====



100%|██████████| 304/304 [00:36<00:00,  8.31it/s] 


              precision    recall  f1-score   support

       False       0.73      0.74      0.73       151
        True       0.74      0.73      0.73       153

    accuracy                           0.73       304
   macro avg       0.73      0.73      0.73       304
weighted avg       0.73      0.73      0.73       304



In [8]:
class ClassificationSubcategoryV3(BaseModel):
    reasoning: str = Field(
        alias="raisonnement",
        description="Raisonnement étape par étape justifiant la classification de l'extrait",
    )
    subcategory: Literal[
        "Faux",
        "Trompeur",
        "Paroles rapportées",
        "Oeuvre de fiction",
        "Rien à signaler",
    ] = Field(alias="sous-catégorie")
    category: Literal["Désinformation", "Pas de désinformation"] = Field(
        alias="catégorie"
    )


async def predict_subcategory_v3(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Vous êtes un assistant IA spécialisé dans l'analyse de contenu médiatique pour détecter la désinformation sur le climat. Votre tâche est d'examiner un extrait textuel de 2 minutes provenant d'une émission télévisée ou radiophonique et de le classer selon la présence ou l'absence de désinformation.

## Extrait à analyser

{text.strip()}

## Classez l'extrait dans l'une des catégories suivantes

Désinformation : L'affirmation est factuellement inexacte, déformée ou exagérée.

Pas de Désinformation :
- Rien à signaler : Aucune désinformation sur le climat n'est présente.
- Paroles rapportées : L'extrait rapporte fidèlement les propos de quelqu'un d'autre.
- Oeuvre de fiction : L'extrait provient d'une oeuvre de fiction.

## Exemples de thématiques susceptibles de contenir de la désinformation sur le climat

- Négation du réchauffement climatique ou mise en doute des preuves de son existence, comme la fonte des glaces ou l'élévation du niveau de la mer.
- Refus d'admettre que l'activité humaine est responsable du changement climatique, préférant attribuer ce phénomène à des causes naturelles.
- Minimisation des impacts du changement climatique, suggérant qu'ils ne seraient pas graves ou même bénéfiques.
- Critique des solutions climatiques, affirmant qu'elles sont inefficaces ou nuisibles.
- Contestation de la validité de la science climatique, la qualifiant d'incertaine ou biaisée.
- Accusations de partialité ou de motivations politiques parmi les scientifiques et les défenseurs du climat.
- Défense de l'utilisation continue des combustibles fossiles pour préserver l'économie et le mode de vie.""",
            }
        ],
        response_format=ClassificationSubcategoryV3,
        timeout=60,
        max_tokens=1000,
        temperature=0,
    )

    classification = ClassificationSubcategoryV3(
        **json.loads(response.choices[0].message.content)
    )
    return classification.category == "Désinformation"


await evaluate(predict_subcategory_v3, train, test)

TEST
====



  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:07<00:00, 10.23it/s]


              precision    recall  f1-score   support

       False       0.70      0.78      0.74        36
        True       0.78      0.71      0.74        41

    accuracy                           0.74        77
   macro avg       0.74      0.74      0.74        77
weighted avg       0.74      0.74      0.74        77



FULL
====



100%|██████████| 304/304 [00:40<00:00,  7.50it/s] 


              precision    recall  f1-score   support

       False       0.72      0.77      0.74       151
        True       0.76      0.71      0.73       153

    accuracy                           0.74       304
   macro avg       0.74      0.74      0.74       304
weighted avg       0.74      0.74      0.74       304



In [9]:
async def predict_deontology(text: str) -> bool:
    response = await acompletion(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""D'après toi, cet extrait radio contient-il de la désinformation sur le sujet du climat ? (oui/non)
Si l'extrait contient de la désinformation sur le climat, un article de fact-checking sera réalisé par un expert humain.
Éxplique le raisonnement de ta classification étape par étape.

# Critères "déontologiques" pour justifier un article de fact-checking

- Importance et intérêt public : La déclaration doit être pertinente et avoir un impact sur l’opinion publique, les politiques, la santé ou les finances.
- Viralité et portée : Elle doit être largement partagée sur les réseaux sociaux, relayée par les médias ou diffusée par des figures influentes.
- Potentiel de nuisance : La déclaration doit présenter des risques ou dangers réels pour la population (ex. : décourager les efforts d’atténuation du changement climatique).
- Falsifiabilité et vérifiabilité : L'affirmation doit être spécifique et vérifiable à l’aide de données crédibles ou d’un consensus scientifique.
- Autorité et influence de l’émetteur : Les déclarations provenant de personnalités publiques, d’officiels ou de grands médias sont prioritaires.
- Clarté et contexte : L’affirmation doit être suffisamment claire pour être analysée et ne pas être sortie de son contexte ou issue de la satire.
- Récurrence et persistance : Si la fausse déclaration revient régulièrement dans le débat public, elle a plus de chances d’être vérifiée.

# Extrait à classifier

{text}""",
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "reponse",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "raisonnement": {
                            "type": "string",
                            "description": "Ton raisonnement pour classifier l'extrait",
                        },
                        "réponse": {
                            "type": "string",
                            "enum": ["oui", "non"],
                            "description": '"oui" si l\'extrait contient de la désinformation sur le climat, sinon "non"',
                        },
                    },
                    "required": ["raisonnement", "réponse"],
                    "additionalProperties": False,
                },
            },
        },
        temperature=0,
    )
    response_dict = json.loads(response.choices[0].message.content)
    return response_dict["réponse"] == "oui"


await evaluate(predict_deontology, train, test)

TEST
====



  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:17<00:00,  4.28it/s]


              precision    recall  f1-score   support

       False       0.64      0.69      0.67        36
        True       0.71      0.66      0.68        41

    accuracy                           0.68        77
   macro avg       0.68      0.68      0.68        77
weighted avg       0.68      0.68      0.68        77



FULL
====



100%|██████████| 304/304 [00:11<00:00, 27.05it/s] 


              precision    recall  f1-score   support

       False       0.74      0.70      0.72       151
        True       0.72      0.75      0.73       153

    accuracy                           0.73       304
   macro avg       0.73      0.73      0.73       304
weighted avg       0.73      0.73      0.73       304



In [10]:
async def predict_cards_v1(text: str) -> bool:
    class ClassificationCards(BaseModel):
        analysis: str = Field(
            description="Ta compréhension et ton analyse de l'extrait de 2 min. Termines ton analyse par l'explication du choix de la catégorie de classification."
        )
        passages: list[str] = Field(
            description="Un ou plusieurs passages de l'extrait qui méritent d'être fact-checké"
        )
        category: Literal[
            "0_accepted",
            "1_its_not_happening",
            "2_humans_are_not_the_cause",
            "3_impacts_are_not_bad",
            "4_solutions_are_ineffective_or_harmful",
            "5_science_is_uncertain",
            "6_advocates_are_biased",
            "7_fossil_fuels_are_needed",
        ] = Field(
            description="La catégorie prédéfinie la plus appropriée pour classifier l'extrait et les passages identifiés. Si plusieurs catégories peuvent correspondre à l'extrait, choisis la catégorie lié au passage le plus clairement climatosceptique."
        )

    system_prompt = """Tu es un expert en désinformation sur les questions environnementales, spécialiste en science climatique et bien informé sur le GIEC. L'utilisateur va te fournir un extrait de deux minutes d'une retranscription d'un programme télévisé ou radiophonique. Certains extraits contiennent des affirmations ou des prises de positions climatosceptiques qui doivent être fact-checker. Ta tâche consiste à identifier ces extraits et à les catégoriser dans différents types de désinformations (1-7). Si tu n'es pas certain que l'extrait contienne du contenu climatosceptique et de la désinformation, classe le en 0_accepted.

# Catégories prédéfinies

- 0_accepted: le passage n'a pas de contenu climatosceptique évident et est accepté tel quel.
- 1_its_not_happening: négation du réchauffement climatique ou allégations selon lesquelles les signes de changement climatique ne seraient pas avérés, tels que la fonte des glaces ou l'élévation du niveau de la mer.
- 2_humans_are_not_the_cause: refus de reconnaître que les gaz à effet de serre produits par l'activité humaine sont la cause du changement climatique, attribuant plutôt cela à des variations naturelles ou autres phénomènes.
- 3_impacts_are_not_bad: minimisation des impacts du changement climatique, suggérant qu'ils pourraient être bénins ou même bénéfiques.
- 4_solutions_are_ineffective_or_harmful: Critique des solutions climatiques, affirmant qu'elles sont nuisibles, inutiles, ou inefficaces.
- 5_science_is_uncertain: contestation de la validité de la science climatique, affirmant qu'elle est incertaine, biaisée ou peu fiable.
- 6_advocates_are_biased: accusations de partialité ou de motivations politiques parmi les scientifiques et les défenseurs de l'action climatique, allant jusqu'à suggérer un alarmisme ou une conspiration.
- 7_fossil_fuels_are_needed: arguments en faveur de l'utilisation continue des combustibles fossiles pour le développement économique et le maintien du niveau de vie.

# Exemples

Voilà quelques exemples de sorties. Nous omettons l'extrait qui sera fourni en entrée par l'utilisateur ainsi que le raisonnement du modèle par brièveté.

{"analysis":"[analysis]","passages":[],"category":"0_accepted"}

{"analysis":"[analysis]","passages":["la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques"],"category":"1_its_not_happening"}

{"analysis":"[analysis]","passages":["il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz"],"category":"2_humans_are_not_the_cause"}

{"analysis":"[analysis]","passages":["il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques"],"category":"3_impacts_are_not_bad"}

{"analysis":"[analysis]","passages":["de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement"],"category":"4_solutions_are_ineffective_or_harmful"}

{"analysis":"[analysis]","passages":["En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée"],"category":"5_science_is_uncertain"}

{"analysis":"[analysis]","passages":["Certains médias et politiciens utilisent la peur du changement climatique pour avancer des agendas politiques partisans, affirmant que les scientifiques amplifient les données pour obtenir plus de subventions."],"category":"6_advocates_are_biased"}

{"analysis":"[analysis]","passages":["L'économie mondiale repose encore largement sur les combustibles fossiles, et leur abandon mettrait en péril notre développement économique et notre sécurité énergétique, car les énergies renouvelables ne sont pas encore prêtes à prendre le relais."],"category":"7_fossil_fuels_are_needed"}
"""
    response = await acompletion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"**Extrait :**\n\n{text.strip()}"},
        ],
        model="gpt-4o-mini",
        max_tokens=1000,
        temperature=0,
        response_format=ClassificationCards,
    )

    # Parse the response
    response_content: str = response.choices[0].message.content
    response_parsed = json.loads(response_content)
    cards_label_pred = response_parsed["category"]
    return cards_label_pred != "0_accepted"


await evaluate(predict_cards_v1, train, test)

TEST
====



100%|██████████| 77/77 [00:11<00:00,  6.46it/s]


              precision    recall  f1-score   support

       False       0.79      0.31      0.44        36
        True       0.60      0.93      0.73        41

    accuracy                           0.64        77
   macro avg       0.69      0.62      0.59        77
weighted avg       0.69      0.64      0.59        77



FULL
====



100%|██████████| 304/304 [00:12<00:00, 23.95it/s] 


              precision    recall  f1-score   support

       False       0.77      0.32      0.45       151
        True       0.57      0.91      0.70       153

    accuracy                           0.62       304
   macro avg       0.67      0.61      0.58       304
weighted avg       0.67      0.62      0.58       304



In [11]:
async def predict_cards_v2(text: str) -> bool:
    response = await acompletion(
        "gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """Tu es un expert en désinformation sur les questions environnementales, spécialiste en science climatique et bien informé sur le GIEC. L'utilisateur va te fournir un extrait de deux minutes d'une retranscription d'un programme télévisé ou radiophonique. Ta mission est de classer cette retranscription dans la catégorie appropriée.

# Catégories prédéfinies

- 0_accepted: Le passage n'a pas de contenu climatosceptique évident et est accepté tel quel.
- 1_its_not_happening: Négation du réchauffement climatique ou allégations selon lesquelles les signes de changement climatique ne seraient pas avérés, tels que la fonte des glaces ou l'élévation du niveau de la mer.
- 2_humans_are_not_the_cause: Refus de reconnaître que les gaz à effet de serre produits par l'activité humaine sont la cause du changement climatique, attribuant plutôt cela à des variations naturelles ou autres phénomènes.
- 3_impacts_are_not_bad: Minimisation des impacts du changement climatique, suggérant qu'ils pourraient être bénins ou même bénéfiques.
- 4_solutions_are_ineffective_or_harmful: Critique des solutions climatiques, affirmant qu'elles sont nuisibles, inutiles, ou inefficaces.
- 5_science_is_uncertain: Contestation de la validité de la science climatique, affirmant qu'elle est incertaine, biaisée ou peu fiable.
- 6_advocates_are_biased: Accusations de partialité ou de motivations politiques parmi les scientifiques et les défenseurs de l'action climatique, allant jusqu'à suggérer un alarmisme ou une conspiration.
- 7_fossil_fuels_are_needed: Arguments en faveur de l'utilisation continue des combustibles fossiles pour le développement économique et le maintien du niveau de vie.

# Exemples

## Exemple 0

<extrait>
les scientifiques observent une hausse des températures mondiales, entraînant des changements climatiques visibles ils soulignent la nécessité d'agir pour en limiter les impacts
</extrait>
<catégorie>0_accepted</catégorie>

## Exemple 1

<extrait>
la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques
</extrait>
</passage climatosceptique>
<catégorie>1_its_not_happening</catégorie>

## Exemple 2

<extrait>
il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz
</extrait>
<catégorie>2_humans_are_not_the_cause</catégorie>

## Exemple 3

<extrait>
il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques
</extrait>
<catégorie>3_impacts_are_not_bad</catégorie>

## Exemple 4

<extrait>
de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement
</extrait>
<catégorie>4_solutions_are_ineffective_or_harmful</catégorie>

## Exemple 5

<extrait>
En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée
</extrait>
<catégorie>5_science_is_uncertain</catégorie>

## Exemple 6

<extrait>
Certains médias et politiciens utilisent la peur du changement climatique pour avancer des agendas politiques partisans, affirmant que les scientifiques amplifient les données pour obtenir plus de subventions.
</extrait>
<catégorie>6_advocates_are_biased</catégorie>

## Exemple 7

<extrait>
L'économie mondiale repose encore largement sur les combustibles fossiles, et leur abandon mettrait en péril notre développement économique et notre sécurité énergétique, car les énergies renouvelables ne sont pas encore prêtes à prendre le relais.
</extrait>
<catégorie>7_fossil_fuels_are_needed</catégorie>""",
            },
            {
                "role": "user",
                "content": f"<extrait>\n{text}\n</extrait>",
            },
        ],
        stop="</",
        temperature=0,
    )
    cards_class = response.choices[0].message.content.split("<catégorie>")[1]
    return cards_class != "0_accepted"


await evaluate(predict_cards_v2, train, test)

TEST
====



  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:04<00:00, 18.10it/s]


              precision    recall  f1-score   support

       False       0.93      0.36      0.52        36
        True       0.63      0.98      0.77        41

    accuracy                           0.69        77
   macro avg       0.78      0.67      0.64        77
weighted avg       0.77      0.69      0.65        77



FULL
====



100%|██████████| 304/304 [00:06<00:00, 48.53it/s] 


              precision    recall  f1-score   support

       False       0.82      0.40      0.54       151
        True       0.61      0.92      0.73       153

    accuracy                           0.66       304
   macro avg       0.71      0.66      0.63       304
weighted avg       0.71      0.66      0.63       304

